In [ ]:
from models.mobilenetv2 import MobileNetV2

model = MobileNetV2(input_shape=(224,224,3), num_classes=38)
model.summary()

import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory

DATASET_PATH = "../data/PlantVillage"

IMAGE_SIZE = (224,224)
BATCH_SIZE = 32

train_ds = image_dataset_from_directory(
    DATASET_PATH,
    validation_split =0.2,
    subset = "training",
    seeds = 123,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE
)

val_ds = image_dataset_from_directory(
    DATASET_PATH,
    validation_split = 0.2,
    subset = "validation",
    seed = 123,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE
)

ModuleNotFoundError: No module named 'models'

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
class_names = train_ds.class_names
len(class_names), class_names

In [ ]:
from models.mobilenetv2 import MobileNetV2

num_classes = len(class_names)

model = MobileNetV2(
    input_shape = (224,224,3),
    num_classes = num_classes
)
model.summary()

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZone(0.1),
])

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

inputs = tf.keras.Input(shape=(224,224,3))
x = data_augmentation(inputs)
x = preprocess_input(x)
outputs = model(x)

train_model = tf.keras.Model(inputs,outputs)

train_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

history = train_model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 10
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss =history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(14,5))

plt.subplot(1,2,1)
plt.plot(epochs_range,acc,labels="Training Accuracy")
plt.plot(epochs_range,val_acc,labels="Validation Accuracy")
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend()
plt.title('Loss')


train_model.save("../models/plant_disease_mobilenetv2.h5")

import numpy as np
from tensorflow.keras.preprocessing import image
img_path = "../data/PlantVillage/Tomato___Late_blight/0a1a3f1b-8b08-4a31-a89e-0dfa45f35bc6___RS_LB 3048.JPG"
img = image.load_img(img_path, target_size=(224,224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array,axis=0)
img_array = preprocess_input(img_array)

pred = train_model.predict(img_array)
class_id = np.argmax(pred)
print("Predicted:",class_names[class_id])